In [ ]:
!pip install pandarallel

In [ ]:
!npm i -g mapshaper@0.5.67

In [ ]:
import geopandas
import pandas
import os
import json
from pandarallel import pandarallel
import topojson as tp
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=20)

In [ ]:
base_dir = './../../datasets/seed/wdpa/wdpa/simp'
f = []
for (dirpath, dirnames, filenames) in os.walk(base_dir):
    for filename in filenames:
        f.append(f'{dirpath}/{filename}')

gdf = pandas.concat([
    geopandas.read_file(file)
    for file in f
]).pipe(geopandas.GeoDataFrame)

In [ ]:
gdf

In [ ]:
base_dir = '.'
f = []
for (dirpath, dirnames, filenames) in os.walk(base_dir):
    for filename in filenames:
        if '.json' in filename and 'r' in filename:
            f.append(f'{dirpath}/{filename}')
gdf = pandas.concat([
    geopandas.read_file(file)
    for file in f
]).pipe(geopandas.GeoDataFrame)

In [ ]:
gdf

In [ ]:
base_dir_in = './../../datasets/seed/wdpa/wdpa/geojson'
f_in = []
for (dirpath, dirnames, filenames) in os.walk(base_dir_in):
    for filename in filenames:
        f_in.append(f'{dirpath}/{filename}')

gdf_in = pandas.concat([
    geopandas.read_file(file)
    for file in f_in
]).pipe(geopandas.GeoDataFrame)

gdf_in

In [ ]:
gdf_in.groupby('IUCN_CAT').agg({'WDPAID': ['count']})

In [ ]:
print(f'{100 - round((196554/200076)*100)}%')

In [ ]:
print(f'{100 - round((200076/254424)*100)}%')

In [ ]:
base_dir_orig = './../../datasets/seed/wdpa/wdpa/data'
f_orig = []
for (dirpath, dirnames, filenames) in os.walk(base_dir_orig):
    for filename in filenames:
        if '.shp' in filename:
            f_orig.append(f'{dirpath}/{filename}')

gdf_orig = pandas.concat([
    geopandas.read_file(file)
    for file in f_orig
]).pipe(geopandas.GeoDataFrame)

gdf_orig

In [ ]:
base_dir_orig = './wdpa_0.shp'
gdf_orig = geopandas.read_file(base_dir_orig)
gdf_orig

In [ ]:
gdf_orig.geometry.sindex

In [ ]:
tp.Topology(data = gdf_orig, prequantize=False).to_svg()

In [ ]:
t = topo.toposimplify(5).to_gdf()

In [ ]:
mapshaper-xl 16gb -i $$f name="$$filename" -split ISO3 -o $@ format=geojson precision=0.0000001;

In [ ]:
def is_valid(geom):
    return geom.is_valid

In [ ]:
gdf_orig_2 = gdf_orig.geometry.parallel_apply(is_valid)

In [ ]:
gdf_orig.columns

In [ ]:
gdf_orig.groupby('geom_is_valid').agg({'WDPAID': ['count']})

In [ ]:
gdf_orig['geom_is_valid']=gdf_orig['geometry'].is_valid

In [ ]:
gdf_orig.groupby('IUCN_CAT').agg({'WDPAID': ['count']})

In [ ]:
test_split = [v for k, v in gdf_orig.groupby('IUCN_CAT')]

In [ ]:
for index, dataset in enumerate(test_split):
    dataset.to_file(f"wdpa_{index}.shp", crs_wkt = {'init': 'epsg:4326', 'no_defs': True})

In [ ]:
gdf_orig.groupby('ISO3').agg({'WDPAID': ['count']})

In [ ]:
from shapely.geometry import box

In [27]:
!mapshaper-xl 16gb -i ./../../datasets/seed/wdpa/wdpa/data/*.shp no-topology combine-files \
    -merge-layers \
    -each "group = Math.trunc(this.id/20000)+1" \
    -split group \
    -o data/ format=geojson precision=0.0001 force ndjson

Allocating 16 GB of heap memory
[o] Wrote data/1.json
[o] Wrote data/2.json
[o] Wrote data/3.json
[o] Wrote data/4.json
[o] Wrote data/5.json
[o] Wrote data/6.json
[o] Wrote data/7.json
[o] Wrote data/8.json
[o] Wrote data/9.json
[o] Wrote data/10.json
[o] Wrote data/11.json


In [ ]:
!mapshaper-xl 16gb -i ./data/r*.json snap combine-files -merge-layers -info

In [28]:
!mapshaper-xl 16gb -i ./data/*.json snap -simplify 20% planar keep-shapes -filter-islands min-vertices=3 min-area=10000m2 remove-empty -filter-slivers min-area=10000m2 remove-empty -clean rewind -o format=geojson force ndjson

Allocating 16 GB of heap memory
[i] Snapped 0 points
[simplify] Repaired 3,403 intersections; 11,536 intersections could not be repaired
[filter-islands] Removed 46,377 islands
[filter-slivers] Removed 22,243 slivers using 0.01+ sqkm variable threshold
[clean] Removed 4,676 / 9,158 slivers using 0.039+ sqkm variable threshold
[clean] Retained 19,298 of 22,066 features
[o] Wrote 10.json
[i] Snapped 0 points
[simplify] Repaired 1,372 intersections; 3,621 intersections could not be repaired
[filter-islands] Removed 1,719 islands
[filter-slivers] Removed 1,086 slivers using 0.01+ sqkm variable threshold
[clean] Removed 2,011 / 2,860 slivers using 0.062+ sqkm variable threshold
[clean] Retained 4,086 of 4,219 features
[o] Wrote 11.json
[i] Snapped 0 points
[simplify] Repaired 2,148 intersections; 77,659 intersections could not be repaired
[filter-islands] Removed 50,711 islands
[filter-slivers] Removed 31,763 slivers using 0.01+ sqkm variable threshold
[clean] Removed 17,423 / 26,375 sliver

In [31]:
!mapshaper-xl 16gb -i ./*.json combine-files -merge-layers -info -o symp-wdpa.shp

Allocating 16 GB of heap memory
[info] 
Layer:    [unnamed layer]
----------------------------------------------
Type:     polygon
Records:  205,434
Bounds:   -180,-85.4119,180,86.4537
CRS:      +proj=longlat +datum=WGS84
Source:   ./10.json

Attribute data
------------+---------------------------------
 Field      | First value
------------+---------------------------------
 CONS_OBJ   | 'Not Applicable'
 DESIG      | 'Private Conservation'
 DESIG_ENG  | 'Private Conservation'
 DESIG_TYPE | 'National'
 GIS_AREA   |         0.184653691756769
 GIS_M_AREA |         0
 GOV_TYPE   | 'Non-profit organisations'
 group      |        10
 INT_CRIT   | 'Not Applicable'
 ISO3       | 'USA'
 IUCN_CAT   | 'V'
 MANG_AUTH  | 'Non-Governmental Organization'
 MANG_PLAN  | 'Not Reported'
 MARINE     | '0'
 METADATAID |      1848
 NAME       | 'Winslow Woods'
 NO_TAKE    | 'Not Applicable'
 NO_TK_AREA |         0
 ORIG_NAME  | 'Winslow Woods'
 OWN_TYPE   | 'Non-profit organisations'
 PA_DEF     | '1'
 PA